## Calling SNPs 

### Using samtools mpileup 

#### First I need to convert my SAM alignment files to BAM files. Then sort and index an la 

In [9]:
!/Applications/bioinfo/samtools1.11/bin/samtools \
samtools view -bS ../analyses/HIPP-DR.sam | samtools sort -o ../analyses/HIPP-DR.bam samtools index ../analyses/HIPP-DR.bam

zsh:1: command not found: samtools
[main] unrecognized command 'samtools'
